In [208]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import statsmodels.api as sm
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta as rd
import schedule
from sklearn.linear_model import LinearRegression
from scipy import stats
import scipy.stats

In [209]:
#function for testing the time it takes for another function to run

from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args,**kw):
        tstart = time()
        result = f(*args,**kw)
        tend = time()
        print('func:%r args: [%r,%r] took: %2.4f sec' % \
              (f.__name__,args,kw,tend-tstart))
        return result
    return wrap
        
        
    

In [210]:
#Inputs
r = 0.06 # Risk-free rate taken from US treasury bonds 
sigma = 0.2
mu = 0.06
seed = None
S0 = 100 #Initial Stock Price
K = 100  #Strike Price
T = 1    #Time to Maturity in Years
N = 4   #Number of Time Steps
u = 1.1  #up-factor in binomial model
d = 1/u  #down-factor in binomial model
optiontype = 'P' #Choose 'C' for call & 'P' for Put
num_paths = 3


In [211]:
def gbm_stock_price(S0, mu, sigma, T, N, seed=None):
    np.random.seed(seed)
    dt = T / N
    t = np.linspace(0, T, N+1)
    W = np.random.standard_normal(size=N+1) 
    W = np.cumsum(W) * np.sqrt(dt)  # Brownian motion
    X = (r - 0.5 * sigma**2) * t + sigma * W  # GBM process
    S = S0 * np.exp(X)  # Stock price process
    S[0] = S0
    return S

In [212]:
#Generates a brownian Motion Matrix W and a Stock Price Matrix X with each row representing 
# a stock price path starting at the initial S0 above

X = np.zeros((num_paths, N))
W = np.zeros((num_paths, N))

np.random.seed(seed)
dt = T / N
t = np.linspace(0, T, N)

for path in range(num_paths):
    # Generate random samples from a normal distribution
    random_samples = np.random.normal(size=N)
    
    # Scale the random samples by the square root of the time step
    scaled_samples = np.sqrt(dt) * random_samples
    
    # Cumulatively sum the scaled samples to simulate Brownian motion
    W[path,:] = np.cumsum(scaled_samples)
    X[path,:] = S0*np.exp((mu - 0.5 * sigma**2) * t + sigma * W[path,:])
X[:,0] = S0


In [213]:
#Visualization of stock price paths in the matrix, can be adjusted properly
X

array([[100.        ,  81.69402767,  82.6751817 ,  71.62839877],
       [100.        , 100.53578577, 100.99793123, 114.87477909],
       [100.        , 120.85766382, 113.51377067, 102.75079923]])

In [214]:
stock_end = X[:, -1]
stock_end

array([ 71.62839877, 114.87477909, 102.75079923])

In [215]:
#determines the payoff at all times except the first time, this may have to be adjusted
# to include the first time as well in case the strike price changes
for i in range(0,N):
    payoffS = np.maximum(K-X,0)
payoffS = payoffS[:, 1:]
payoffS

array([[18.30597233, 17.3248183 , 28.37160123],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [217]:
value_end = np.maximum(K-stock_end,0)
value_end

array([28.37160123,  0.        ,  0.        ])

In [219]:


# Calculate time step
dt = T / N

# Initialize a list to store matrices for each time step
probability_matrices = []

# Calculate conditional probabilities for each time step
for j in range(N-2, -1, -1):
    probability_matrix = np.zeros((num_paths, num_paths))

    for k in range(num_paths):
        # Calculate the conditional probability using the log-normal distribution
        mu_conditional = np.log(X[:, j+1] / X[k, j]) / dt - 0.5 * sigma**2
        sigma_conditional = sigma * np.sqrt(dt)

        # Calculate the probability using the log-normal distribution formula
        prob = scipy.stats.norm.pdf(np.log(X[:, j+1] / X[k, j]), loc=mu_conditional, scale=sigma_conditional)

        # Normalize the probabilities to sum to 1
        prob /= np.sum(prob)

        # Store the probability in the matrix
        probability_matrix[k, :] = prob

    # Append the probability matrix for the current time step to the list
    probability_matrices.append(probability_matrix)

# Now, probability_matrices[j] represents the matrix of probabilities for time step j


In [221]:
payoffS

array([[18.30597233, 17.3248183 , 28.37160123],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [222]:
#Organized such that the first row of any matrix is the transition probabilities from the top node,
# to all nodes in the next time step, and then goes down the column of nodes repeating process for each row
#matrix 0 in list corresponds to time step J-1
probability_matrices

[array([[9.99946920e-01, 1.28004632e-16, 5.30804612e-05],
        [1.10413819e-24, 1.28443367e-03, 9.98715566e-01],
        [2.30612060e-43, 9.93762610e-01, 6.23739034e-03]]),
 array([[9.99999994e-01, 5.70275859e-09, 5.09776884e-21],
        [1.01073986e-08, 9.97329475e-01, 2.67051463e-03],
        [5.84745324e-29, 1.46485060e-06, 9.99998535e-01]]),
 array([[2.98825320e-09, 9.99999701e-01, 2.96119646e-07],
        [2.98825320e-09, 9.99999701e-01, 2.96119646e-07],
        [2.98825320e-09, 9.99999701e-01, 2.96119646e-07]])]

In [226]:
#multiplies the first through third row of each matrix by payoff of the corresponding time 
#ex: in matrix 1, the first row will be multiplied by the last row of the payoff matrix
#these will then be added up and put in the top right of the continuation matrix (2,2), 
# this is based on how I calculated the probabilites earlier with matrix 1 corresponding to the probabilities for time j to j+1

k = 0
cont_matrix = np.zeros((num_paths,num_paths))
for j in range(num_paths-1,-1,-1):
    for i in range(num_paths):
        cont_matrix[i][j] = np.dot(probability_matrices[k][i,:],payoffS[:,j])
    k+=1

In [227]:
cont_matrix

array([[5.47028805e-08, 1.73248182e+01, 2.83700953e+01],
       [5.47028805e-08, 1.75108845e-07, 3.13261684e-23],
       [5.47028805e-08, 1.01306065e-27, 6.54283340e-42]])

In [228]:
#take max of the continuation value and current payoff (missing e^-rdt factor)
max_value = np.maximum(cont_matrix,payoffS)
max_value

array([[1.83059723e+01, 1.73248183e+01, 2.83716012e+01],
       [5.47028805e-08, 1.75108845e-07, 3.13261684e-23],
       [5.47028805e-08, 1.01306065e-27, 6.54283340e-42]])

In [229]:
#after the above process which goes backwards through the paths, we should have a final price value by discounting the expectations at time 1 I believe.

price = sum(1/3*max_value[:,0])
price 

6.10199081208825